In [6]:
import pandas as pd
import requests
import ipywidgets as widgets
import time
from IPython.display import display

In [7]:
# Cargar el dataset
file_path = 'Limpieza/data/INFORMACIÓN_TRIMESTRAL_DE_ACCESOS_FIJOS_A_INTERNET_POR_PROVEEDOR,_DEPARTAMENTO,_MUNICIPIO,_SEGMENTO,_TECNOLOGIA,_Y_VELOCIDAD_DE_CONEXIÓN_COMPLETO.csv'
df = pd.read_csv(file_path)

In [8]:
# Filtrar los municipios únicos
municipios_unicos = df[['MUNICIPIO', 'DEPARTAMENTO']].drop_duplicates()

In [9]:
coordenadas_path = 'Limpieza/data/DatasetCoordenadas.csv'

In [10]:
df_coordenadas = pd.read_csv(coordenadas_path)

In [11]:
print(df_coordenadas)

             Ciudad    Latitud   Longitud
0            SOACHA   4.582774 -74.212592
1             NEIVA   2.935250 -75.281013
2    MOSQUERA (CUN)   4.709222 -74.226543
3              SOPO   4.908816 -73.940157
4            MADRID   4.734119 -74.263045
..              ...        ...        ...
322     CAÑASGORDAS   6.749890 -76.025390
323        EL PEÑOL   1.453650 -77.440170
324    LA PAZ (CES)  10.384390 -73.173320
325        FILANDIA   4.674720 -75.658330
326         PIVIJAY  10.461670 -74.616210

[327 rows x 3 columns]


In [12]:
# Renombrar la columna Ciudad si es necesario
df_coordenadas.rename(columns={'Ciudad': 'MUNICIPIO'}, inplace=True)

In [13]:
# Realizar la unión (merge) de los dos DataFrames
df_municipios_con_coordenadas = pd.merge(municipios_unicos, df_coordenadas, on='MUNICIPIO', how='left')

In [14]:
df_municipios_con_coordenadas

,MUNICIPIO,DEPARTAMENTO,Latitud,Longitud
0,CALI,VALLE DEL CAUCA,NaN,NaN
1,AMALFI,ANTIOQUIA,6.910160,-75.077640
2,BARBOSA,ANTIOQUIA,NaN,NaN
3,BELLO,ANTIOQUIA,6.336309,-75.560973
4,CAROLINA,ANTIOQUIA,NaN,NaN
...,...,...,...,...
1116,PUERTO COLOMBIA,GUAINÍA,NaN,NaN
1117,PANA PANA,GUAINÍA,NaN,NaN
1118,PAPUNAHUA,VAUPÉS,NaN,NaN
1119,MIRITI - PARANÁ,AMAZONAS,NaN,NaN


In [15]:
municipios_unicos

,MUNICIPIO,DEPARTAMENTO
0,CALI,VALLE DEL CAUCA
3,AMALFI,ANTIOQUIA
6,BARBOSA,ANTIOQUIA
16,BELLO,ANTIOQUIA
18,CAROLINA,ANTIOQUIA
...,...,...
85280,PUERTO COLOMBIA,GUAINÍA
89765,PANA PANA,GUAINÍA
209419,PAPUNAHUA,VAUPÉS
1776386,MIRITI - PARANÁ,AMAZONAS


In [16]:
# Verificar cuántos municipios no tienen coordenadas
print(df_municipios_con_coordenadas[df_municipios_con_coordenadas['Latitud'].isna()].shape)

(926, 4)


In [17]:
# Eliminar las filas donde Latitud o Longitud tengan valores nulos
df_municipios_sin_nulos = df_municipios_con_coordenadas.dropna(subset=['Latitud', 'Longitud'])

# Mostrar las primeras filas para verificar
print(df_municipios_sin_nulos.head())

# Verificar el número de filas después de eliminar los nulos
print(f"Número de municipios con coordenadas: {df_municipios_sin_nulos.shape[0]}")


     MUNICIPIO DEPARTAMENTO   Latitud   Longitud
1       AMALFI    ANTIOQUIA  6.910160 -75.077640
3        BELLO    ANTIOQUIA  6.336309 -75.560973
15     CALARCA      QUINDÍO  4.526779 -75.642213
18    FILANDIA      QUINDÍO  4.674720 -75.658330
20  LA TEBAIDA      QUINDÍO  4.453516 -75.788988
Número de municipios con coordenadas: 195


In [18]:
df_municipios_sin_nulos

,MUNICIPIO,DEPARTAMENTO,Latitud,Longitud
1,AMALFI,ANTIOQUIA,6.910160,-75.077640
3,BELLO,ANTIOQUIA,6.336309,-75.560973
15,CALARCA,QUINDÍO,4.526779,-75.642213
18,FILANDIA,QUINDÍO,4.674720,-75.658330
20,LA TEBAIDA,QUINDÍO,4.453516,-75.788988
...,...,...,...,...
1091,SEVILLA,VALLE DEL CAUCA,4.264250,-75.930850
1092,CISNEROS,ANTIOQUIA,5.253970,-75.154030
1095,COROZAL,SUCRE,9.318470,-75.293300
1101,VALDIVIA,ANTIOQUIA,5.016100,-75.628540


In [19]:
import folium

In [20]:
# Crear un mapa centrado en Colombia con límites establecidos
mapa = folium.Map(location=[4.570868, -74.297333], zoom_start=6, max_bounds=True)

In [21]:
# Definir los límites de Colombia (aproximados)
bounds = [[-4.2316872, -81.7281111], [13.4, -66.85]]  # Coordenadas aproximadas de la esquina inferior izquierda y superior derecha de Colombia

In [22]:
# Establecer los límites
mapa.fit_bounds(bounds)
mapa.options = {
    'dragging': False,  # Deshabilitar arrastrar
    'zoomControl': True,  # Mantener el control de zoom
    'scrollWheelZoom': False,  # Deshabilitar zoom con scroll
    'doubleClickZoom': False,  # Deshabilitar zoom con doble clic
}

In [23]:
# Agregar los marcadores al mapa
for index, row in df_municipios_sin_nulos.iterrows():
    folium.Marker(
        location=[row['Latitud'], row['Longitud']],
        popup=f"{row['MUNICIPIO']}, {row['DEPARTAMENTO']}",
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(mapa)

In [24]:
    # Guardar el mapa en un archivo HTML
mapa.save('mapa_colombia_limitado.html')

In [25]:
df.columns

Index(['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CÓDIGO DANE', 'DEPARTAMENTO',
       'CÓDIGO DANE.1', 'MUNICIPIO', 'SEGMENTO', 'TECNOLOGÍA',
       'VELOCIDAD BAJADA', 'VELOCIDAD SUBIDA', 'No. ACCESOS FIJOS A INTERNET'],
      dtype='object')

In [26]:
df_unido = pd.merge(df, df_municipios_sin_nulos, how='left', on='MUNICIPIO')

In [27]:
df_unido_limpio = df_unido.dropna(subset=['Latitud', 'Longitud'])

In [28]:
df_unido_limpio

,AÑO,TRIMESTRE,PROVEEDOR,CÓDIGO DANE,DEPARTAMENTO_x,CÓDIGO DANE.1,MUNICIPIO,SEGMENTO,TECNOLOGÍA,VELOCIDAD BAJADA,VELOCIDAD SUBIDA,No. ACCESOS FIJOS A INTERNET,DEPARTAMENTO_y,Latitud,Longitud
3,2021,3,@DIGITAL GROUP SAS,5,ANTIOQUIA,5031,AMALFI,CORPORATIVO,FIBER TO THE HOME (FTTH),20.0,5.0,1,ANTIOQUIA,6.910160,-75.077640
4,2021,3,@DIGITAL GROUP SAS,5,ANTIOQUIA,5031,AMALFI,CORPORATIVO,FIBER TO THE HOME (FTTH),30.0,5.0,1,ANTIOQUIA,6.910160,-75.077640
5,2021,3,@DIGITAL GROUP SAS,5,ANTIOQUIA,5031,AMALFI,RESIDENCIAL - ESTRATO 1,FIBER TO THE HOME (FTTH),18.0,5.0,1,ANTIOQUIA,6.910160,-75.077640
16,2021,3,@DIGITAL GROUP SAS,5,ANTIOQUIA,5088,BELLO,RESIDENCIAL - ESTRATO 2,FIBER TO THE HOME (FTTH),30.0,5.0,3,ANTIOQUIA,6.336309,-75.560973
17,2021,3,@DIGITAL GROUP SAS,5,ANTIOQUIA,5088,BELLO,RESIDENCIAL - ESTRATO 2,FIBER TO THE HOME (FTTH),8.0,4.0,10,ANTIOQUIA,6.336309,-75.560973
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964165,2024,1,ZUMA COMUNICACIONES DE COLOMBIA S.A.S,25,CUNDINAMARCA,25793,TAUSA,RESIDENCIAL - ESTRATO 1,WIFI,2300.0,690.0,2,CUNDINAMARCA,5.195847,-73.887223
1964166,2024,1,ZUMA COMUNICACIONES DE COLOMBIA S.A.S,25,CUNDINAMARCA,25793,TAUSA,RESIDENCIAL - ESTRATO 1,WIFI,4300.0,1290.0,5,CUNDINAMARCA,5.195847,-73.887223
1964167,2024,1,ZUMA COMUNICACIONES DE COLOMBIA S.A.S,25,CUNDINAMARCA,25793,TAUSA,RESIDENCIAL - ESTRATO 1,WIFI,7300.0,2190.0,1,CUNDINAMARCA,5.195847,-73.887223
1964168,2024,1,ZUMA COMUNICACIONES DE COLOMBIA S.A.S,25,CUNDINAMARCA,25793,TAUSA,RESIDENCIAL - ESTRATO 2,WIFI,4300.0,1290.0,10,CUNDINAMARCA,5.195847,-73.887223


In [29]:
df_unido_limpio.columns

Index(['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CÓDIGO DANE', 'DEPARTAMENTO_x',
       'CÓDIGO DANE.1', 'MUNICIPIO', 'SEGMENTO', 'TECNOLOGÍA',
       'VELOCIDAD BAJADA', 'VELOCIDAD SUBIDA', 'No. ACCESOS FIJOS A INTERNET',
       'DEPARTAMENTO_y', 'Latitud', 'Longitud'],
      dtype='object')

In [30]:
df

,AÑO,TRIMESTRE,PROVEEDOR,CÓDIGO DANE,DEPARTAMENTO,CÓDIGO DANE.1,MUNICIPIO,SEGMENTO,TECNOLOGÍA,VELOCIDAD BAJADA,VELOCIDAD SUBIDA,No. ACCESOS FIJOS A INTERNET
0,2021,3,SERVICE WELL NET S.A.S,76,VALLE DEL CAUCA,76001,CALI,RESIDENCIAL - ESTRATO 2,FIBER TO THE HOME (FTTH),3.0,1.5,3
1,2021,3,SERVICE WELL NET S.A.S,76,VALLE DEL CAUCA,76001,CALI,RESIDENCIAL - ESTRATO 2,FIBER TO THE HOME (FTTH),2.0,1.0,12
2,2021,3,SERVICE WELL NET S.A.S,76,VALLE DEL CAUCA,76001,CALI,RESIDENCIAL - ESTRATO 2,FIBER TO THE HOME (FTTH),4.0,2.0,7
3,2021,3,@DIGITAL GROUP SAS,5,ANTIOQUIA,5031,AMALFI,CORPORATIVO,FIBER TO THE HOME (FTTH),20.0,5.0,1
4,2021,3,@DIGITAL GROUP SAS,5,ANTIOQUIA,5031,AMALFI,CORPORATIVO,FIBER TO THE HOME (FTTH),30.0,5.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1964330,2024,1,ZUMA COMUNICACIONES DE COLOMBIA S.A.S,25,CUNDINAMARCA,25899,ZIPAQUIRÁ,RESIDENCIAL - ESTRATO 1,FIBER TO THE HOME (FTTH),80300.0,80300.0,2
1964331,2024,1,ZUMA COMUNICACIONES DE COLOMBIA S.A.S,25,CUNDINAMARCA,25899,ZIPAQUIRÁ,SIN ESTRATIFICAR,FIBER TO THE HOME (FTTH),220300.0,220300.0,14
1964332,2024,1,ZUMA COMUNICACIONES DE COLOMBIA S.A.S,25,CUNDINAMARCA,25899,ZIPAQUIRÁ,RESIDENCIAL - ESTRATO 3,FIBER TO THE HOME (FTTH),60300.0,60300.0,27
1964333,2024,1,ZUMA COMUNICACIONES DE COLOMBIA S.A.S,25,CUNDINAMARCA,25899,ZIPAQUIRÁ,RESIDENCIAL - ESTRATO 2,FIBER TO THE HOME (FTTH),80300.0,80300.0,26


In [31]:
# Mantener solo una columna de DEPARTAMENTO (suponiendo que quieras mantener la de df_proveedores)
df_unido_limpio = df_unido_limpio.drop(columns=['DEPARTAMENTO_y'])

In [32]:
# Opcionalmente puedes renombrar la columna restante a 'DEPARTAMENTO'
df_unido_limpio = df_unido_limpio.rename(columns={'DEPARTAMENTO_x': 'DEPARTAMENTO'})


In [33]:
# Verificar que las columnas estén correctas
df_unido_limpio

,AÑO,TRIMESTRE,PROVEEDOR,CÓDIGO DANE,DEPARTAMENTO,CÓDIGO DANE.1,MUNICIPIO,SEGMENTO,TECNOLOGÍA,VELOCIDAD BAJADA,VELOCIDAD SUBIDA,No. ACCESOS FIJOS A INTERNET,Latitud,Longitud
3,2021,3,@DIGITAL GROUP SAS,5,ANTIOQUIA,5031,AMALFI,CORPORATIVO,FIBER TO THE HOME (FTTH),20.0,5.0,1,6.910160,-75.077640
4,2021,3,@DIGITAL GROUP SAS,5,ANTIOQUIA,5031,AMALFI,CORPORATIVO,FIBER TO THE HOME (FTTH),30.0,5.0,1,6.910160,-75.077640
5,2021,3,@DIGITAL GROUP SAS,5,ANTIOQUIA,5031,AMALFI,RESIDENCIAL - ESTRATO 1,FIBER TO THE HOME (FTTH),18.0,5.0,1,6.910160,-75.077640
16,2021,3,@DIGITAL GROUP SAS,5,ANTIOQUIA,5088,BELLO,RESIDENCIAL - ESTRATO 2,FIBER TO THE HOME (FTTH),30.0,5.0,3,6.336309,-75.560973
17,2021,3,@DIGITAL GROUP SAS,5,ANTIOQUIA,5088,BELLO,RESIDENCIAL - ESTRATO 2,FIBER TO THE HOME (FTTH),8.0,4.0,10,6.336309,-75.560973
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964165,2024,1,ZUMA COMUNICACIONES DE COLOMBIA S.A.S,25,CUNDINAMARCA,25793,TAUSA,RESIDENCIAL - ESTRATO 1,WIFI,2300.0,690.0,2,5.195847,-73.887223
1964166,2024,1,ZUMA COMUNICACIONES DE COLOMBIA S.A.S,25,CUNDINAMARCA,25793,TAUSA,RESIDENCIAL - ESTRATO 1,WIFI,4300.0,1290.0,5,5.195847,-73.887223
1964167,2024,1,ZUMA COMUNICACIONES DE COLOMBIA S.A.S,25,CUNDINAMARCA,25793,TAUSA,RESIDENCIAL - ESTRATO 1,WIFI,7300.0,2190.0,1,5.195847,-73.887223
1964168,2024,1,ZUMA COMUNICACIONES DE COLOMBIA S.A.S,25,CUNDINAMARCA,25793,TAUSA,RESIDENCIAL - ESTRATO 2,WIFI,4300.0,1290.0,10,5.195847,-73.887223


In [34]:
# Crear el mapa centrado en Colombia
m = folium.Map(location=[4.570868, -74.297333], zoom_start=6)

In [37]:
# Crear un dropdown para seleccionar el municipio
municipios = df_unido_limpio['MUNICIPIO'].unique()
municipio_selector = widgets.Dropdown(
    options=municipios,
    value=municipios[0],
    description='Municipio:',
    disabled=False,
)

In [38]:
# Crear un slider para seleccionar el número de accesos
accesos_slider = widgets.IntSlider(
    value=10,
    min=0,
    max=df_unido_limpio['No. ACCESOS FIJOS A INTERNET'].max(),
    step=10,
    description='No. Accesos:',
)

In [44]:
# Función para actualizar el mapa
def actualizar_mapa(municipio, accesos):
    # Filtrar el DataFrame según el municipio y el número de accesos
    df_filtrado = df_unido_limpio[(df_unido_limpio['MUNICIPIO'] == municipio) &
                                  (df_unido_limpio['No. ACCESOS FIJOS A INTERNET'] >= accesos)]

    # Crear un nuevo mapa centrado en Colombia
    mapa = folium.Map(location=[4.570868, -74.297333], zoom_start=6)

    # Agregar los puntos en el mapa
    for idx, row in df_filtrado.iterrows():
        folium.Marker([row['Latitud'], row['Longitud']],
                      popup=f"Proveedor: {row['PROVEEDOR']}<br>Tecnología: {row['TECNOLOGÍA']}<br>Accesos: {row['No. ACCESOS FIJOS A INTERNET']}").add_to(mapa)

    # Guardar el mapa en un archivo HTML, sobreescribiendo cada vez
    mapa.save('mapa_interactivo.html')
    print(f'Mapa guardado como mapa_interactivo.html')

In [45]:
# Función para actualizar el mapa al cambiar las opciones
def actualizar_mapa_y_guardar(municipio, accesos):
    actualizar_mapa(municipio, accesos)
    display(municipio_selector, accesos_slider)

In [48]:
df_unido_limpio.to_csv('df_unificado_limpio.csv', index=False)